In [1]:
import os
from glob import glob

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from contextlib import contextmanager
from time import time
import matplotlib.pyplot as plt
import seaborn as sns
import random
import shutil

%matplotlib inline


# ref: Kaggleコード遺産 https://qiita.com/kaggle_grandmaster-arai-san/items/d59b2fb7142ec7e270a5 
class Timer:
    def __init__(self, logger=None, format_str="{:.3f}[s]", prefix=None, suffix=None, sep=" "):

        if prefix: format_str = str(prefix) + sep + format_str
        if suffix: format_str = format_str + sep + str(suffix)
        self.format_str = format_str
        self.logger = logger
        self.start = None
        self.end = None

    @property
    def duration(self):
        if self.end is None:
            return 0
        return self.end - self.start

    def __enter__(self):
        self.start = time()

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.end = time()
        out_str = self.format_str.format(self.duration)
        if self.logger:
            self.logger.info(out_str)
        else:
            print(out_str)


def seed_everything(seed: int):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)

# 再現性確保!
seed_everything(510)

In [2]:
# 学習用のログデータと正解ラベル
train_log_df = pd.read_csv("./input/train_log.csv")
train_label_df = pd.read_csv("./input/train_label.csv")

# 宿のデータ
yado_df = pd.read_csv("./input/yado.csv")

sample_submission_df = pd.read_csv("./input/sample_submission.csv")

In [3]:
train_log_df.head()

,session_id,seq_no,yad_no
0,000007603d533d30453cc45d0f3d119f,0,2395
1,0000ca043ed437a1472c9d1d154eb49b,0,13535
2,0000d4835cf113316fe447e2f80ba1c8,0,123
3,0000fcda1ae1b2f431e55a7075d1f500,0,8475
4,000104bdffaaad1a1e0a9ebacf585f33,0,96


In [4]:
train_label_df.head()

,session_id,yad_no
0,000007603d533d30453cc45d0f3d119f,4101
1,0000ca043ed437a1472c9d1d154eb49b,8253
2,0000d4835cf113316fe447e2f80ba1c8,4863
3,0000fcda1ae1b2f431e55a7075d1f500,1652
4,000104bdffaaad1a1e0a9ebacf585f33,96


In [5]:
yado_df.head()

,yad_no,yad_type,total_room_cnt,wireless_lan_flg,onsen_flg,kd_stn_5min,kd_bch_5min,kd_slp_5min,kd_conv_walk_5min,wid_cd,ken_cd,lrg_cd,sml_cd
0,1,0,129.0,1.0,0,1.0,NaN,NaN,1.0,f0112abf369fb03cdc5f5309300913da,072c85e1653e10c9c7dd065ad007125a,449c52ef581d5f9ef311189469a0520e,677a32689cd1ad74e867f1fbe43a3e1c
1,2,0,23.0,1.0,0,NaN,NaN,NaN,NaN,d86102dd9c232bade9a97dccad40df48,b4d2fb4e51ea7bca80eb1270aa474a54,5c9a8f48e9df0234da012747a02d4b29,4ee16ee838dd2703cc9a1d5a535f0ced
2,3,0,167.0,1.0,1,1.0,NaN,NaN,1.0,46e33861f921c3e38b81998fbf283f01,572d60f0f5212aacda515ebf81fb0a3a,8a623b960557e87bd1f4edf71b6255be,ab9480fd72a44d51690ab16c4ad4d49c
3,4,0,144.0,1.0,0,1.0,NaN,NaN,1.0,46e33861f921c3e38b81998fbf283f01,107c7305a74c8dcc4f143de208bf7ec2,52c9ea83f2cfe92be54cb6bc961edf21,1cc3e1838bb0fd0fde0396130b1f82b9
4,5,0,41.0,1.0,1,NaN,NaN,NaN,NaN,43875109d1dab93592812c50d18270a7,75617bb07a2785a948ab1958909211f1,9ea5a911019b66ccd42f556c42a2fe2f,be1b876af18afc4deeb3081591d2a910


In [6]:
# train_logに情報をmerge
train_label_df["label"] = 1
train_log_df = train_log_df.merge(train_label_df, how="left", on=["session_id", "yad_no"])
train_log_df["label"] = train_log_df["label"].fillna(0)
train_log_df =train_log_df.merge(yado_df, how="left", on="yad_no")
train_log_df.head()

,session_id,seq_no,yad_no,label,yad_type,total_room_cnt,wireless_lan_flg,onsen_flg,kd_stn_5min,kd_bch_5min,kd_slp_5min,kd_conv_walk_5min,wid_cd,ken_cd,lrg_cd,sml_cd
0,000007603d533d30453cc45d0f3d119f,0,2395,0.0,0,113.0,1.0,0,NaN,NaN,NaN,NaN,dc414a17890cfc17d011d5038b88ca93,d78f53d0856617bc782f02c3280dfef2,4fd631b15116098340cdb099c86a5a40,4044dac1931ddaa5a967e09506d76343
1,0000ca043ed437a1472c9d1d154eb49b,0,13535,0.0,0,40.0,1.0,0,1.0,NaN,NaN,1.0,b07b75d367ebece55a23ceecc939fff4,0a66f6ab9c0507059da6f22a0e1f1690,9ab5718fd88c6e5f9fec37a51827d428,7aff71bb47acb796d425c5ed5e6dfb3f
2,0000d4835cf113316fe447e2f80ba1c8,0,123,0.0,0,17.0,1.0,0,NaN,NaN,NaN,NaN,46e33861f921c3e38b81998fbf283f01,572d60f0f5212aacda515ebf81fb0a3a,dac434451fe9bd50068191f41fe792e3,b7c56c5d2855b39366b4ebe9a4eded93
3,0000fcda1ae1b2f431e55a7075d1f500,0,8475,0.0,0,65.0,1.0,0,1.0,NaN,NaN,1.0,46e33861f921c3e38b81998fbf283f01,107c7305a74c8dcc4f143de208bf7ec2,3a6cd37aa9e38fd96d9dafc2615643d0,f2fcbd8e62872147efde0acef474e1f2
4,000104bdffaaad1a1e0a9ebacf585f33,0,96,1.0,0,228.0,1.0,0,NaN,NaN,NaN,1.0,e9316013ee1b03f4525fe361c46ce9c5,84efa50e52f9b471c95bfc3b21b854ad,a1370d90ed3b80ee41311bbbab46aec9,d72674f02c5340d90f245e3177727650


In [7]:
# yadoのtrainにおける予約回数をカウントし、宿情報をmerge
yado_score = train_log_df.query("label==1")["yad_no"].value_counts().reset_index()
yado_df = yado_score.merge(yado_df, how="left", on='yad_no')

yado_df.head()

,yad_no,count,yad_type,total_room_cnt,wireless_lan_flg,onsen_flg,kd_stn_5min,kd_bch_5min,kd_slp_5min,kd_conv_walk_5min,wid_cd,ken_cd,lrg_cd,sml_cd
0,3338,426,0,703.0,1.0,0,1.0,NaN,NaN,NaN,46e33861f921c3e38b81998fbf283f01,572d60f0f5212aacda515ebf81fb0a3a,8a623b960557e87bd1f4edf71b6255be,1d9f09b9e2bd43cebc9885a46388739a
1,12350,358,0,696.0,1.0,0,NaN,NaN,NaN,NaN,46e33861f921c3e38b81998fbf283f01,572d60f0f5212aacda515ebf81fb0a3a,8a623b960557e87bd1f4edf71b6255be,1d9f09b9e2bd43cebc9885a46388739a
2,10095,302,0,2007.0,1.0,0,NaN,NaN,NaN,1.0,46e33861f921c3e38b81998fbf283f01,572d60f0f5212aacda515ebf81fb0a3a,8a623b960557e87bd1f4edf71b6255be,f7b42d92528e7a88617c4b26e033d3e5
3,719,250,0,600.0,1.0,0,1.0,NaN,NaN,1.0,f0112abf369fb03cdc5f5309300913da,072c85e1653e10c9c7dd065ad007125a,ed62e66a5031c23c78bd03ccf9f3ef70,d3d1cf557f10fadb1fbc0b429bf14578
4,8553,247,0,550.0,1.0,0,NaN,NaN,NaN,1.0,46e33861f921c3e38b81998fbf283f01,572d60f0f5212aacda515ebf81fb0a3a,8a623b960557e87bd1f4edf71b6255be,1d9f09b9e2bd43cebc9885a46388739a


In [8]:
train_candidate_df = train_log_df[["session_id", "seq_no", "yad_no"]]
train_candidate_df = train_candidate_df.drop_duplicates()
train_candidate_df = train_candidate_df.sort_values(["session_id",  "seq_no", "yad_no"]).reset_index(drop=True)
train_candidate_df["is_sml_cd_mode"] = 0
train_candidate_df.head(10)

,session_id,seq_no,yad_no,is_sml_cd_mode
0,000007603d533d30453cc45d0f3d119f,0,2395,0
1,0000ca043ed437a1472c9d1d154eb49b,0,13535,0
2,0000d4835cf113316fe447e2f80ba1c8,0,123,0
3,0000fcda1ae1b2f431e55a7075d1f500,0,8475,0
4,000104bdffaaad1a1e0a9ebacf585f33,0,96,0
5,000104bdffaaad1a1e0a9ebacf585f33,1,898,0
6,00011afe25c343301ee961b3b0ce2e4d,0,6868,0
7,000125c737df1802b6e365f93c96d3c8,0,8602,0
8,0001763050a10b21062a1304fb743fd4,0,13106,0
9,000178c4d4d567d4715331dd0cdab76c,0,12062,0


In [9]:
# session_idごとに、最も出現頻度の高いsml_cdを集計しておく
train_sml_cd_mode_df = train_log_df.groupby('session_id')['sml_cd'].agg(lambda x: x.mode().iloc[0]).reset_index()
train_sml_cd_mode_df.head()

,session_id,sml_cd
0,000007603d533d30453cc45d0f3d119f,4044dac1931ddaa5a967e09506d76343
1,0000ca043ed437a1472c9d1d154eb49b,7aff71bb47acb796d425c5ed5e6dfb3f
2,0000d4835cf113316fe447e2f80ba1c8,b7c56c5d2855b39366b4ebe9a4eded93
3,0000fcda1ae1b2f431e55a7075d1f500,f2fcbd8e62872147efde0acef474e1f2
4,000104bdffaaad1a1e0a9ebacf585f33,d72674f02c5340d90f245e3177727650


In [10]:
tmp_df = train_sml_cd_mode_df.merge(yado_df[["yad_no", "sml_cd"]], how="left", on='sml_cd')
tmp_df = tmp_df.drop(columns="sml_cd")
tmp_df["seq_no"] = -1
tmp_df["is_sml_cd_mode"] = 1
tmp_df.head(50)

,session_id,yad_no,seq_no,is_sml_cd_mode
0,000007603d533d30453cc45d0f3d119f,11101,-1,1
1,000007603d533d30453cc45d0f3d119f,7157,-1,1
2,000007603d533d30453cc45d0f3d119f,11134,-1,1
3,000007603d533d30453cc45d0f3d119f,4391,-1,1
4,000007603d533d30453cc45d0f3d119f,8927,-1,1
5,000007603d533d30453cc45d0f3d119f,12837,-1,1
6,000007603d533d30453cc45d0f3d119f,12846,-1,1
7,000007603d533d30453cc45d0f3d119f,5821,-1,1
8,000007603d533d30453cc45d0f3d119f,997,-1,1
9,000007603d533d30453cc45d0f3d119f,1648,-1,1


In [11]:
train_candidate_df = pd.concat([train_candidate_df, tmp_df], ignore_index=True)
train_candidate_df = train_candidate_df.sort_values(["session_id", "yad_no"]).reset_index(drop=True)
train_candidate_df.head(50)

,session_id,seq_no,yad_no,is_sml_cd_mode
0,000007603d533d30453cc45d0f3d119f,-1,997,1
1,000007603d533d30453cc45d0f3d119f,-1,1168,1
2,000007603d533d30453cc45d0f3d119f,-1,1648,1
3,000007603d533d30453cc45d0f3d119f,-1,2008,1
4,000007603d533d30453cc45d0f3d119f,-1,2134,1
5,000007603d533d30453cc45d0f3d119f,0,2395,0
6,000007603d533d30453cc45d0f3d119f,-1,2395,1
7,000007603d533d30453cc45d0f3d119f,-1,2528,1
8,000007603d533d30453cc45d0f3d119f,-1,2808,1
9,000007603d533d30453cc45d0f3d119f,-1,2995,1


In [12]:
# train_candidate_df = train_candidate_df.groupby(['session_id', 'yad_no'])[['seq_no', 'is_sml_cd_mode']].max()
# train_candidate_df.head(50)

In [13]:
train_candidate_df = train_candidate_df.merge(yado_df, how="inner", on="yad_no")
train_candidate_df = train_candidate_df.sort_values(["session_id", "yad_no"]).reset_index(drop=True)
train_candidate_df.head()

,session_id,seq_no,yad_no,is_sml_cd_mode,count,yad_type,total_room_cnt,wireless_lan_flg,onsen_flg,kd_stn_5min,kd_bch_5min,kd_slp_5min,kd_conv_walk_5min,wid_cd,ken_cd,lrg_cd,sml_cd
0,000007603d533d30453cc45d0f3d119f,-1,997,1,9,0,250.0,1.0,0,1.0,NaN,NaN,NaN,dc414a17890cfc17d011d5038b88ca93,d78f53d0856617bc782f02c3280dfef2,4fd631b15116098340cdb099c86a5a40,4044dac1931ddaa5a967e09506d76343
1,000007603d533d30453cc45d0f3d119f,-1,1168,1,4,0,138.0,1.0,0,1.0,NaN,NaN,1.0,dc414a17890cfc17d011d5038b88ca93,d78f53d0856617bc782f02c3280dfef2,4fd631b15116098340cdb099c86a5a40,4044dac1931ddaa5a967e09506d76343
2,000007603d533d30453cc45d0f3d119f,-1,1648,1,8,0,180.0,1.0,0,1.0,NaN,NaN,1.0,dc414a17890cfc17d011d5038b88ca93,d78f53d0856617bc782f02c3280dfef2,4fd631b15116098340cdb099c86a5a40,4044dac1931ddaa5a967e09506d76343
3,000007603d533d30453cc45d0f3d119f,-1,2008,1,1,0,36.0,1.0,1,NaN,1.0,NaN,NaN,dc414a17890cfc17d011d5038b88ca93,d78f53d0856617bc782f02c3280dfef2,4fd631b15116098340cdb099c86a5a40,4044dac1931ddaa5a967e09506d76343
4,000007603d533d30453cc45d0f3d119f,-1,2134,1,3,0,18.0,1.0,1,NaN,1.0,NaN,NaN,dc414a17890cfc17d011d5038b88ca93,d78f53d0856617bc782f02c3280dfef2,4fd631b15116098340cdb099c86a5a40,4044dac1931ddaa5a967e09506d76343


In [14]:
# 正解ラベルに含まれているレコードの index を配列で取得して
target_index = pd.merge(train_candidate_df.reset_index(), train_label_df, on=["session_id", "yad_no"], how="inner")["index"].values

# 正解Indexに含まれている場合 1 / そうでないと 0 のラベルを作成
train_candidate_df["target"] = train_candidate_df.index.isin(target_index).astype(int)

# 見た目を揃えるために session / yad でソートしておく
train_candidate_df = train_candidate_df.sort_values(["session_id", "yad_no"]).reset_index(drop=True)

train_candidate_df.head(50)

,session_id,seq_no,yad_no,is_sml_cd_mode,count,yad_type,total_room_cnt,wireless_lan_flg,onsen_flg,kd_stn_5min,kd_bch_5min,kd_slp_5min,kd_conv_walk_5min,wid_cd,ken_cd,lrg_cd,sml_cd,target
0,000007603d533d30453cc45d0f3d119f,-1,997,1,9,0,250.0,1.0,0,1.0,NaN,NaN,NaN,dc414a17890cfc17d011d5038b88ca93,d78f53d0856617bc782f02c3280dfef2,4fd631b15116098340cdb099c86a5a40,4044dac1931ddaa5a967e09506d76343,0
1,000007603d533d30453cc45d0f3d119f,-1,1168,1,4,0,138.0,1.0,0,1.0,NaN,NaN,1.0,dc414a17890cfc17d011d5038b88ca93,d78f53d0856617bc782f02c3280dfef2,4fd631b15116098340cdb099c86a5a40,4044dac1931ddaa5a967e09506d76343,0
2,000007603d533d30453cc45d0f3d119f,-1,1648,1,8,0,180.0,1.0,0,1.0,NaN,NaN,1.0,dc414a17890cfc17d011d5038b88ca93,d78f53d0856617bc782f02c3280dfef2,4fd631b15116098340cdb099c86a5a40,4044dac1931ddaa5a967e09506d76343,0
3,000007603d533d30453cc45d0f3d119f,-1,2008,1,1,0,36.0,1.0,1,NaN,1.0,NaN,NaN,dc414a17890cfc17d011d5038b88ca93,d78f53d0856617bc782f02c3280dfef2,4fd631b15116098340cdb099c86a5a40,4044dac1931ddaa5a967e09506d76343,0
4,000007603d533d30453cc45d0f3d119f,-1,2134,1,3,0,18.0,1.0,1,NaN,1.0,NaN,NaN,dc414a17890cfc17d011d5038b88ca93,d78f53d0856617bc782f02c3280dfef2,4fd631b15116098340cdb099c86a5a40,4044dac1931ddaa5a967e09506d76343,0
5,000007603d533d30453cc45d0f3d119f,0,2395,0,7,0,113.0,1.0,0,NaN,NaN,NaN,NaN,dc414a17890cfc17d011d5038b88ca93,d78f53d0856617bc782f02c3280dfef2,4fd631b15116098340cdb099c86a5a40,4044dac1931ddaa5a967e09506d76343,0
6,000007603d533d30453cc45d0f3d119f,-1,2395,1,7,0,113.0,1.0,0,NaN,NaN,NaN,NaN,dc414a17890cfc17d011d5038b88ca93,d78f53d0856617bc782f02c3280dfef2,4fd631b15116098340cdb099c86a5a40,4044dac1931ddaa5a967e09506d76343,0
7,000007603d533d30453cc45d0f3d119f,-1,2528,1,5,0,10.0,1.0,1,NaN,NaN,NaN,NaN,dc414a17890cfc17d011d5038b88ca93,d78f53d0856617bc782f02c3280dfef2,4fd631b15116098340cdb099c86a5a40,4044dac1931ddaa5a967e09506d76343,0
8,000007603d533d30453cc45d0f3d119f,-1,2808,1,6,0,128.0,1.0,0,1.0,NaN,NaN,NaN,dc414a17890cfc17d011d5038b88ca93,d78f53d0856617bc782f02c3280dfef2,4fd631b15116098340cdb099c86a5a40,4044dac1931ddaa5a967e09506d76343,0
9,000007603d533d30453cc45d0f3d119f,-1,2995,1,5,0,24.0,NaN,1,NaN,1.0,NaN,NaN,dc414a17890cfc17d011d5038b88ca93,d78f53d0856617bc782f02c3280dfef2,4fd631b15116098340cdb099c86a5a40,4044dac1931ddaa5a967e09506d76343,0


In [15]:
from sklearn.preprocessing import LabelEncoder

In [16]:
le = LabelEncoder()

# train_candidate_df = train_candidate_df.drop(columns="session_id")
train_candidate_df["wid_cd"] = le.fit_transform(train_candidate_df["wid_cd"])
train_candidate_df["ken_cd"] = le.fit_transform(train_candidate_df["ken_cd"])
train_candidate_df["lrg_cd"] = le.fit_transform(train_candidate_df["lrg_cd"])
train_candidate_df["sml_cd"] = le.fit_transform(train_candidate_df["sml_cd"])

In [17]:
train_candidate_df.head(50)

,session_id,seq_no,yad_no,is_sml_cd_mode,count,yad_type,total_room_cnt,wireless_lan_flg,onsen_flg,kd_stn_5min,kd_bch_5min,kd_slp_5min,kd_conv_walk_5min,wid_cd,ken_cd,lrg_cd,sml_cd,target
0,000007603d533d30453cc45d0f3d119f,-1,997,1,9,0,250.0,1.0,0,1.0,NaN,NaN,NaN,9,44,92,151,0
1,000007603d533d30453cc45d0f3d119f,-1,1168,1,4,0,138.0,1.0,0,1.0,NaN,NaN,1.0,9,44,92,151,0
2,000007603d533d30453cc45d0f3d119f,-1,1648,1,8,0,180.0,1.0,0,1.0,NaN,NaN,1.0,9,44,92,151,0
3,000007603d533d30453cc45d0f3d119f,-1,2008,1,1,0,36.0,1.0,1,NaN,1.0,NaN,NaN,9,44,92,151,0
4,000007603d533d30453cc45d0f3d119f,-1,2134,1,3,0,18.0,1.0,1,NaN,1.0,NaN,NaN,9,44,92,151,0
5,000007603d533d30453cc45d0f3d119f,0,2395,0,7,0,113.0,1.0,0,NaN,NaN,NaN,NaN,9,44,92,151,0
6,000007603d533d30453cc45d0f3d119f,-1,2395,1,7,0,113.0,1.0,0,NaN,NaN,NaN,NaN,9,44,92,151,0
7,000007603d533d30453cc45d0f3d119f,-1,2528,1,5,0,10.0,1.0,1,NaN,NaN,NaN,NaN,9,44,92,151,0
8,000007603d533d30453cc45d0f3d119f,-1,2808,1,6,0,128.0,1.0,0,1.0,NaN,NaN,NaN,9,44,92,151,0
9,000007603d533d30453cc45d0f3d119f,-1,2995,1,5,0,24.0,NaN,1,NaN,1.0,NaN,NaN,9,44,92,151,0


In [18]:
# test
test_log_df = pd.read_csv("./input/test_log.csv")

test_candidate_df = test_log_df[["session_id", "seq_no", "yad_no"]]
test_candidate_df = test_candidate_df.drop_duplicates()
test_candidate_df = test_candidate_df.sort_values(["session_id",  "seq_no", "yad_no"]).reset_index(drop=True)
test_candidate_df["is_sml_cd_mode"] = 0

# session_idごとに、最も出現頻度の高いsml_cdを集計しておく
test_log_df =test_log_df.merge(yado_df[["yad_no","sml_cd"]], how="inner", on="yad_no")
test_sml_cd_mode_df = test_log_df.groupby('session_id')['sml_cd'].agg(lambda x: x.mode().iloc[0]).reset_index()

tmp_df = test_sml_cd_mode_df.merge(yado_df[["yad_no", "sml_cd"]], how="left", on='sml_cd')
tmp_df = tmp_df.drop(columns="sml_cd")
tmp_df["seq_no"] = -1
tmp_df["is_sml_cd_mode"] = 1

test_candidate_df = pd.concat([test_candidate_df, tmp_df], ignore_index=True)
test_candidate_df = test_candidate_df.sort_values(["session_id", "yad_no"]).reset_index(drop=True)

test_candidate_df = test_candidate_df.merge(yado_df, how="inner", on="yad_no")
test_candidate_df = test_candidate_df.sort_values(["session_id", "yad_no"]).reset_index(drop=True)

# test_candidate_df = test_candidate_df.drop(columns="session_id")
test_candidate_df["wid_cd"] = le.fit_transform(test_candidate_df["wid_cd"])
test_candidate_df["ken_cd"] = le.fit_transform(test_candidate_df["ken_cd"])
test_candidate_df["lrg_cd"] = le.fit_transform(test_candidate_df["lrg_cd"])
test_candidate_df["sml_cd"] = le.fit_transform(test_candidate_df["sml_cd"])

test_candidate_df.head(50)

,session_id,seq_no,yad_no,is_sml_cd_mode,count,yad_type,total_room_cnt,wireless_lan_flg,onsen_flg,kd_stn_5min,kd_bch_5min,kd_slp_5min,kd_conv_walk_5min,wid_cd,ken_cd,lrg_cd,sml_cd
0,00001149e9c73985425197104712478c,-1,930,1,5,0,408.0,1.0,0,1.0,NaN,NaN,1.0,3,5,98,298
1,00001149e9c73985425197104712478c,-1,1063,1,1,0,60.0,1.0,0,NaN,NaN,NaN,NaN,3,5,98,298
2,00001149e9c73985425197104712478c,1,1959,0,27,0,173.0,1.0,0,1.0,NaN,NaN,1.0,3,5,98,298
3,00001149e9c73985425197104712478c,-1,1959,1,27,0,173.0,1.0,0,1.0,NaN,NaN,1.0,3,5,98,298
4,00001149e9c73985425197104712478c,-1,2680,1,25,0,150.0,1.0,0,1.0,NaN,NaN,1.0,3,5,98,298
5,00001149e9c73985425197104712478c,-1,2811,1,16,0,49.0,1.0,0,1.0,NaN,NaN,1.0,3,5,98,298
6,00001149e9c73985425197104712478c,-1,2911,1,11,0,105.0,1.0,0,1.0,NaN,NaN,1.0,3,5,98,298
7,00001149e9c73985425197104712478c,0,3560,0,14,0,205.0,1.0,0,NaN,NaN,NaN,1.0,3,5,98,298
8,00001149e9c73985425197104712478c,-1,3560,1,14,0,205.0,1.0,0,NaN,NaN,NaN,1.0,3,5,98,298
9,00001149e9c73985425197104712478c,-1,4420,1,11,0,124.0,1.0,0,1.0,NaN,NaN,1.0,3,5,98,298
